In [1]:
import sys
sys.path.append('..')
import os
from os.path import expanduser

import lcatools

In [2]:
from lcatools.tools import archive_from_json

In [3]:
elcd3_file = os.path.join(os.path.expanduser('~'),'Dropbox', 'data', 'ELCD', 'ELCD3.2.zip')
catalog_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'lca-tools-datafiles', 'catalogs')
lcia_file = os.path.join(catalog_dir, 'elcd_lcia.json.gz')

In [15]:
def lcia(process, quantity):
    print(', '.join([quantity[desc] for desc in ('Method', 'Category', 'Indicator')]))
    print('-'*60)
    score = 0.0
    for x in process.exchanges():
        if x.flow.cf(quantity) != 0:
            c = x.flow.cf(quantity)
            v = x.value
            score += c*v
            print('%10.3g x %-10.3g = %-10.3g %s' % (c, v, c*v, x.flow['Name']) )
    print('='*60)
    print('             Total score: %g [%s]' % (score, quantity.reference_entity.unitstring()))
    return score

### ELCD LCIA Computation

In [4]:
l = archive_from_json(lcia_file)

Loading JSON data from /home/b/GitHub/lca-tools-datafiles/catalogs/elcd_lcia.json.gz:
Found Extension: zip
**Upstream reference encountered: /home/b/Dropbox/data/ELCD/ELCD3.2.zip

54 new quantity entities added (54 total)
1256 new flow entities added (1256 total)
0 new process entities added (0 total)


In [5]:
from lcatools.providers.ilcd import IlcdArchive

In [6]:
ps = IlcdArchive(elcd3_file, upstream=l)

Found Extension: zip


In [7]:
ps.retrieve_or_fetch_entity(ps.list_objects('Process')[27])

In [8]:
p = _

In [9]:
qs = (x for x in l.quantities() if 'Method' in x.keys())

In [10]:
q = next(qs)

In [11]:
q.keys()

dict_keys(['Comment', 'Method', 'Name', 'origin', 'Category', 'ReferenceYear', 'Duration', 'Indicator', 'externalId'])

In [23]:
%time lcia(p, next(qs))

ILCD2011, Ionizing radiation, Comparative Toxic Unit for ecosystems (CTUe) expressing an estimate of the potentially affected fraction of species (PAF) integrated over time and volume per unit mass of a radionucleide emitted (PAF m3 year/kg).
------------------------------------------------------------
  1.86e-05 x 0.00089    = 1.65e-08   antimony-124
  1.51e-05 x 0.0273     = 4.12e-07   antimony-125
  1.04e-05 x 0.639      = 6.65e-06   iodine-131
  4.59e-09 x 133        = 6.13e-07   hydrogen-3
  2.43e-05 x 0.144      = 3.51e-06   cobalt-58
  4.62e-05 x 0.0242     = 1.12e-06   antimony-124
   5.5e-05 x 0.000267   = 1.47e-08   cesium-134
  0.000124 x 0.0275     = 3.41e-06   cesium-134
  2.26e-05 x 0.000968   = 2.19e-08   cobalt-60
  4.78e-06 x 9.97e-10   = 4.77e-15   manganese-54
  4.49e-05 x 0.000355   = 1.59e-08   cesium-137
  0.000172 x 0.0207     = 3.56e-06   carbon-14
  1.37e-05 x 0.0295     = 4.04e-07   silver-110
  6.83e-05 x 4.24e-06   = 2.9e-10    tellurium-123
  5.41e-05 x 0.1

0.0031144941582810586

### Ecoinvent LCIA Computation

In [5]:
e = archive_from_json(os.path.join(catalog_dir, 'ei_lcia.json.gz'))

Loading JSON data from /home/b/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz:
**Upstream reference encountered: /data/LCI/Ecoinvent/3.2/undefined

708 new quantity entities added (708 total)
3255 new flow entities added (3255 total)
0 new process entities added (0 total)


In [6]:
from lcatools.providers.ecospold2 import EcospoldV2Archive

In [7]:
ei_32_path = os.path.join(os.path.expanduser('~'),'Dropbox', 'data', 'Ecoinvent', 
                               '3.2')
ei_32_undefined = os.path.join(ei_32_path, 'current_Version_3.2_undefined_ecoSpold02.7z')


In [8]:
E = EcospoldV2Archive(ei_32_undefined, upstream=e, prefix='datasets - public')

Found Extension: 7z


In [9]:
p = E.retrieve_or_fetch_entity(E.list_datasets()[2790])

In [10]:
qe = (x for x in e.quantities() if 'Method' in x.keys())  # or if isLciaMethod

In [11]:
[str(x) for x in p.exchanges()]

[' Input:     [0.106 [m3]] Water, cooling, unspecified natural origin (CAS 007732-18-5) [natural resource, in water]',
 ' Input:     [1e-05 [kg]] chlorine, gaseous (CAS 7782-50-5) [By-product classification: allocatable product]',
 ' Input:     [0.0893 [kg]] lignite [By-product classification: allocatable product]',
 ' Input:     [0.006 [kg]] water, completely softened, from decarbonised water, at user [By-product classification: allocatable product]',
 ' Input:     [4.92e-13 [unit]] lignite power plant [By-product classification: allocatable product]',
 ' Input:     [0.00223 [kg]] SOx retained, in lignite flue gas desulfurisation [By-product classification: allocatable product]',
 ' Input:     [0.15 [kg]] water, decarbonised, at user [By-product classification: allocatable product]',
 'Output:     [2.14e-10 [kg]] Strontium (CAS 007440-24-6) [air, non-urban air or from high stacks]',
 'Output:     [1e-09 [kg]] PAH, polycyclic aromatic hydrocarbons (CAS 130498-29-2) [air, non-urban air 

In [18]:
lcia(p, next(qe))

ReCiPe Midpoint (H) w/o LT, terrestrial acidification w/o LT, TAP100 w/o LT
------------------------------------------------------------
         1 x 0.000198   = 0.000198   Sulfur dioxide
      0.56 x 0.000165   = 9.24e-05   Nitrogen oxides
             Total score: 0.0002904 [kg SO2-Eq]


0.0002904

 same system model three different ways:

In [19]:
E_fg = EcospoldV2Archive(os.path.join(ei_32_path, 'current_Version_3.2_apos_ecoSpold02.7z'), 
                         upstream=e, prefix='datasets')

Found Extension: 7z


In [20]:
E_bg = EcospoldV2Archive(os.path.join(ei_32_path, 'current_Version_3.2_apos_lci_ecoSpold02.7z'), 
                         upstream=e, prefix='datasets')

Found Extension: 7z


In [21]:
E_lcia = EcospoldV2Archive(os.path.join(ei_32_path, 'current_Version_3.2_apos_lcia_ecoSpold02.7z'), 
                           upstream=e, prefix='datasets')
for q in e.quantities():
    E_lcia.add(q)

Found Extension: 7z


In [23]:
file = E_fg.list_datasets()[28]

In [24]:
p_fg = E_fg.retrieve_or_fetch_entity(file)

In [25]:
len([x for x in p_fg.exchanges()])

130

In [26]:
p_bg = E_bg.retrieve_or_fetch_entity(file)

In [27]:
len([x for x in p_bg.exchanges()])

1879

In [28]:
f_lcia = E_lcia.retrieve_lcia_scores(file)

In [36]:
q = next(qe)

In [37]:
lcia(p_fg, q)

IMPACT 2002+ (Endpoint), human health, respiratory effects (inorganics)
------------------------------------------------------------
    0.0986 x 0.000105   = 1.04e-05   Particulates, < 2.5 um
   0.00769 x 0.0019     = 1.46e-05   Sulfur dioxide
     0.012 x 1.35e-10   = 1.61e-12   Ammonia
    0.0125 x 0.00212    = 2.67e-05   Nitrogen oxides
  0.000103 x 0.00222    = 2.28e-07   Carbon monoxide, fossil
  0.000103 x 2.55e-06   = 2.63e-10   Carbon monoxide, non-fossil
             Total score: 5.18897e-05 [points]


5.188967521736182e-05

In [38]:
lcia(p_bg, q)
print('aggregated result: %g' %f_lcia.cf(q))

IMPACT 2002+ (Endpoint), human health, respiratory effects (inorganics)
------------------------------------------------------------
   0.00769 x 0.00191    = 1.47e-05   Sulfur dioxide
     0.012 x 7.84e-08   = 9.39e-10   Ammonia
   0.00769 x 9.24e-07   = 7.1e-09    Sulfur dioxide
    0.0125 x 3.38e-06   = 4.24e-08   Nitrogen oxides
    0.0125 x 8.06e-13   = 1.01e-14   Nitrogen oxides
    0.0125 x 1.69e-06   = 2.12e-08   Nitrogen oxides
  0.000103 x 7.26e-07   = 7.47e-11   Carbon monoxide, fossil
  0.000103 x 0.00233    = 2.39e-07   Carbon monoxide, fossil
    0.0986 x 0.000106   = 1.04e-05   Particulates, < 2.5 um
  0.000103 x 2.01e-06   = 2.07e-10   Carbon monoxide, fossil
    0.0986 x 1.38e-07   = 1.37e-08   Particulates, < 2.5 um
     0.012 x 7.09e-09   = 8.49e-11   Ammonia
   0.00615 x 9.68e-13   = 5.96e-15   Sulfur trioxide
   0.00769 x 3.49e-06   = 2.68e-08   Sulfur dioxide
     0.012 x 5.35e-12   = 6.41e-14   Ammonia
  0.000103 x 6.06e-09   = 6.24e-13   Carbon monoxide, non-fos

### Okay, now sugarbeets

In [22]:
ei_foreground = archive_from_json(os.path.join(catalog_dir, 'ecoinvent_3.2_apos_xlsx.json.gz'))

Loading JSON data from /home/b/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_apos_xlsx.json.gz:
**Upstream reference encountered: /home/b/Dropbox/data/Ecoinvent/3.2/activity_overview_for_users_3.2_undefined.xlsx

718 new quantity entities added (718 total)
6767 new flow entities added (6767 total)
11420 new process entities added (11420 total)


In [23]:
results = ei_foreground.search(Name=['sugar','beet'])

In [24]:
def show(search):
    for i, k in enumerate(search):
        print('%3d: %s' % (i, k))

In [25]:
show(results)

  0: market for sugar, from sugar beet [GLO]
  1: sugar beet pulp [Intermediate flow]
  2: market for sugar beet [GLO]
  3: ethanol production from sugar beet [CH]
  4: field application of vinasse, from fermentation of sugar beet [GLO]
  5: market for vinasse, from fermentation of sugar beet [GLO]
  6: ethanol production from sugar beet molasses [RoW]
  7: sugar beet seed production, for sowing [GLO]
  8: sugar beet pulp to generic market for energy feed [GLO]
  9: market for sugar beet pulp [GLO]
 10: molasses, from sugar beet [Intermediate flow]
 11: sugar, from sugar beet [Intermediate flow]
 12: sugar beet production [CH]
 13: sugar beet seed, for sowing [Intermediate flow]
 14: market for sugar beet seed, for sowing [GLO]
 15: ethanol production from sugar beet [RoW]
 16: vinasse, from fermentation of sugar beet molasses [Intermediate flow]
 17: market for molasses, from sugar beet [GLO]
 18: ethanol production from sugar beet molasses [CH]
 19: market for vinasse, from fermentat

In [26]:
E_fg.list_datasets(results[8].get_uuid())

['b2c33b9b-1091-4bed-92a3-decec3b8bd78_7cb72345-4338-4f2d-830f-65bba3530fdb.spold',
 'b2c33b9b-1091-4bed-92a3-decec3b8bd78_8b2dfb58-660d-44eb-b83f-5d9e28e467db.spold',
 'b2c33b9b-1091-4bed-92a3-decec3b8bd78_e98c4987-0c27-4ad8-aa06-568bea573460.spold']

In [27]:
for f in E_fg.list_datasets(results[8].get_uuid()):
    E_fg.retrieve_or_fetch_entity(f)

In [28]:
[x.reference_entity for x in E_fg.processes('b2c33')]

[{<lcatools.exchanges.Exchange at 0x7f67a6944518>,
  <lcatools.exchanges.Exchange at 0x7f67a69446a0>}]

In [29]:
p = E_fg['b2c33b9b-1091-4bed-92a3-decec3b8bd78']

In [30]:
pulp_to_energy = list(p.reference_entity)

In [31]:
pulp_to_energy[2].flow.serialize(characterizations=True)

{'CasNumber': None,
 'Comment': 'EcoSpold02 Flow',
 'Compartment': ['By-product classification: allocatable product'],
 'Name': 'protein feed, 100% crude',
 'characterizations': [{'isReference': True,
   'quantity': '487df68b-4994-4027-8fdc-a4dc298257b7'}],
 'entityId': '8b2dfb58-660d-44eb-b83f-5d9e28e467db',
 'entityType': 'flow',
 'externalId': '8b2dfb58-660d-44eb-b83f-5d9e28e467db',
 'origin': '/home/b/Dropbox/data/Ecoinvent/3.2/current_Version_3.2_apos_ecoSpold02.7z'}

In [32]:
[str(x) for x in p.exchanges()]

[' Input:     [-0.00185 [kg]] molasses, from sugar beet [By-product classification: allocatable product] (#)',
 ' Input:     [0.00335 [kg]] Carbon dioxide, in air (CAS 000124-38-9) [natural resource, in air]',
 ' Input:     [-8.95e-05 [m3]] wastewater, from residence [By-product classification: Waste] (#)',
 ' Input:     [-0.104 [kg]] vinasse, from fermentation of sugar beet [By-product classification: Recyclable] (#)',
 ' Input:     [7.46e-12 [unit]] ethanol fermentation plant [By-product classification: allocatable product] (#)',
 ' Input:     [8.07e-13 [unit]] sugar refinery [By-product classification: allocatable product] (#)',
 ' Input:     [8.79e-09 [m3]] Water, cooling, unspecified natural origin (CAS 007732-18-5) [natural resource, in water]',
 ' Input:     [-0.00188 [kg]] limestone residue [By-product classification: Waste] (#)',
 ' Input:     [0.000506 [kg]] sulfuric acid [By-product classification: allocatable product] (#)',
 ' Input:     [0.00149 [kg]] lime [By-product clas

In [33]:
def exch_to_spold(exch):
    return '%s_%s.spold' % (exch.process.get_uuid(), exch.flow.get_uuid())

In [34]:
p_lcia = E_lcia.retrieve_lcia_scores(exch_to_spold(pulp_to_energy[0]))

In [37]:
_cf = (str(x) for x in p_lcia.characterizations())

In [43]:
next(_cf)

'energy feed, gross [By-product classification: allocatable product]: [0.000293 [points]] LC quantity: ReCiPe Endpoint (H,A) w/o LT, ecosystem quality w/o LT, climate change, ecosystems w/o LT'

In [44]:
%time o = E_lcia.objectify(exch_to_spold(pulp_to_energy[1]))

CPU times: user 810 ms, sys: 2.5 s, total: 3.31 s
Wall time: 10.2 s


In [45]:
%time cfs = E_lcia._collect_impact_scores(o)

CPU times: user 50 ms, sys: 0 ns, total: 50 ms
Wall time: 68.2 ms


In [46]:
len(cfs)

701

In [50]:
def tagslist():
    tags = dict()
    for q in E_lcia.quantities():
        if 'Method' in q.keys():
            tags[q['Name']] = q
    return tags

In [51]:
%time tags = tagslist()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.05 ms


NameError: name 'spold_reference_flow' is not defined